# 3D Point Cloud Model of COVID-19

This project will create a 3D point cloud visualization model of the COVID-19 pandemic. The model will focus on modeling the global spatial patterns along with the statistics of different types of live cases for each country by thousands or even millions of georeferenced points.

## PREPARATION

Check python module/package installation. Uncomment the following code cell and run the specific `pip` statement first to install the missing python module/package if any module error occurs through `import` step.

In [ ]:
# pip install open3d

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import json
import open3d as o3d
import urllib
import time
import os
from matplotlib import cm
from math import cos, sin, pi
from matplotlib import colors

## STEP1: LAND MASK DATA

#### Notice: 
The following code cells are commented because the local file `landmask.geojson` has already created so that you can literally jump this step rather than run it again. But you are welcomed to uncomment and try again if needed.

In [ ]:
# # create 1-D arrays for lon and lat
# grid_lons = np.arange(-180, 180, 1)
# grid_lats = np.arange(-90, 90, 1)
# # cartesian product to pair each lon and lat
# grid_coords = [[a, b] for a in grid_lons for b in grid_lats]

In [ ]:
# # create a temporary dictionary for pandas dataframe
# coords_lon = []
# coords_lat = []
# dict_temp = {'lon': coords_lon, 'lat': coords_lat}

# for coord in grid_coords:
#     coords_lon.append(coord[0])
#     coords_lat.append(coord[1])

In [ ]:
# points = pd.DataFrame.from_dict(dict_temp)
# # generate the geometry from lon/lat columns
# points = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy(points.lon, points.lat))
# polygons = gpd.read_file('World_Boundaries/World_Countries_(Generalized).shp')

# # keep the same coordinate reference system
# points = points.set_crs('EPSG:4326')
# polygons = polygons.to_crs('EPSG:4326')

In [ ]:
# # spatial join
# land = gpd.sjoin(points, polygons, how='inner', op='intersects')
# land = land.drop(columns=['lon', 'lat', 'index_right'])

In [ ]:
# # write geojson
# land.to_file('landmask.geojson', driver='GeoJSON')

## STEP2: COVID-19 DATA

#### Notice: 
The following code cells are commented because the local file `iso_coords.json` has already created so that you can literally jump this step rather than run it again. But you are welcomed to uncomment and try again if needed.

**2.1 extract existing countries' `ISO` and the corresponding `geometry` from the `landmask.geojson` file**

In [ ]:
# # extract iso and coords value only
# iso_coords = {}

# with open('landmask.geojson', 'r') as f:
#     data = json.load(f)
    
# features = data['features']

# for feature in features:
#     iso = feature['properties']['ISO']
#     coords = feature['geometry']['coordinates']
    
#     # check key exist or not
#     if iso not in iso_coords.keys():
#         # initial key
#         iso_coords[iso] = [coords]
#     else:
#         iso_coords[iso].append(coords)


In [ ]:
# # write iso and coords value into local file
# with open('iso_coords.json', 'w') as f:
#     json.dump(iso_coords, f)

**2.2 request available countries kept tracking on the COVID-19 API**

In [ ]:
# # query all possible country names 
# url_countries = 'https://api.covid19api.com/countries'
# response = urllib.request.urlopen(url_countries)
# all_countries = json.loads(response.read())

**2.3 extract existing countries' `slug` for requesting COVID-19 data by country**


In [ ]:
# # extract iso-slug pairs for all existing countries
# iso_slugs = {}

# for i in range(len(all_countries)):
#     iso = all_countries[i]['ISO2']
#     if iso in iso_coords.keys():
#         iso_slugs[iso] = all_countries[i]['Slug']

**2.4 start requesting each country's COVID-19 data and store in local JSON file**


In [ ]:
# # utc time rather than local time
# action_date = time.strftime('%Y%m%d', time.gmtime())

# # same request url part
# request = 'https://api.covid19api.com/total/country/'

In [ ]:
# for iso, slug in iso_slugs.items():
#     # request
#     req = request + slug
#     res = urllib.request.urlopen(req)
#     data = json.loads(res.read())

#     # modify geometry
#     ## loop each dict in the list
#     ## delete lat/lon/ keys
#     ## copy and paste geometry from iso_coords dict
#     for i in range(len(data)):
#         # delete keys
#         data[i].pop('CountryCode')
#         data[i].pop('Province')
#         data[i].pop('City')
#         data[i].pop('CityCode')
#         data[i].pop('Lat')
#         data[i].pop('Lon')
        
#         # add keys
#         data[i]['ISO'] = iso
#         data[i]['Geometry'] = iso_coords[iso]

#     # store locally
#     filename = f'data_by_country/{slug}_{action_date}.json'
#     with open(filename, 'w') as f:
#         json.dump(data, f)


## STEP3: MODEL DEVELOPMENT

**3.1 read local covid-19 data for each country**

In [ ]:
# coords with active cases for all countries
covid_coords = []

# loop over each country/file
for file in os.listdir('data_by_country'):
    with open(f'data_by_country/{file}', 'r') as f:
        data = json.load(f)
        
        # extract latest date
        # considering the empty list
        if data:
            latest_data = data[-1]
            # extract coords and active
            geom = latest_data['Geometry']
            active = latest_data['Active']
            for coords in geom:
                coords.append(active)
                covid_coords.append(coords)   

**3.2 write 3D ply model**

In [3]:
# Convert the coordinates from lon/lat to x/y/z.
def coords_conversion(lon, lat, num):
    ''' Convert the coordinates from lon/lat to x/y/z. '''
    O = [0, 0, 0]   # center
    R = 6400        # radius
    S = 0.01        # scale
    
    
    # calculate the radian of the sphere
    rad_lat, rad_lon = lat * pi / 180, lon * pi / 180
    
    # calculate the cartesian coordiantes of each point
    x = O[0] + S * R * cos(rad_lat) * cos(rad_lon)
    y = O[1] + S * R * cos(rad_lat) * sin(rad_lon)
    z = O[2] + S * R * sin(rad_lat)


    return (x, y, z)

In [135]:
def point_color(x, min_val, max_val):                                     
    
    # convert the input parameters from string to float
    min_val = float(min_val)
    max_val = float(max_val)
    
    # calculate the rank of the number
    ratio = (x-min_val) / (max_val-min_val)
    # calculate the value of color according to the ratio
    cmap = cm.YlOrRd
    rgb = cmap(int(ratio*255))
    r = int(rgb[0]*255)
    g = int(rgb[1]*255)
    b = int(rgb[2]*255)

    return (r, g, b)
    print('##### EQ colored #####')

In [8]:
# write ply file for 3D model.
def model(path):
    ''' write ply file for 3D model'''
    with open(path, 'w') as fw:
    # write headers with required format 
        fw.write('ply\nformat ascii 1.0\n')
        fw.write('element vertex %d\n' % len(model_coords))
        fw.write('property float x\n')
        fw.write('property float y\n')
        fw.write('property float z\n')

        if len(model_colors) == len(model_coords):
            fw.write('property uchar red\n')
            fw.write('property uchar green\n')
            fw.write('property uchar blue\n')

        fw.write('end_header\n')
        
        # write data
        if len(model_colors) == len(model_coords):
            for coord, color in zip(model_coords, model_colors):
                fw.write("%f %f %f %d %d %d\n" % (
                    coord[0],
                    coord[1],
                    coord[2],
                    color[0],
                    color[1],
                    color[2]
                    ))
        else:
            for coord in model_coords:
                fw.write("%f %f %f\n" % (
                    coord[0],
                    coord[1],
                    coord[2]
                    ))
        print('##### PLY model created #####')


In [136]:
# find vmin and vmax
covid_coords_t = np.transpose(covid_coords)
covid_active = covid_coords_t[2]

max_active = np.amax(covid_active)
min_active = np.amin(covid_active)

q1 = np.percentile(covid_active, [25, 50, 75])[0]
q2 = np.percentile(covid_active, [25, 50, 75])[1]
q3 = np.percentile(covid_active, [25, 50, 75])[2]

qr = q3 - q1

# normal values
nor_min = q1 - 1.5 * qr
nor_max = q3 + 1.5 * qr


# observation values
if nor_min < np.amin(covid_active):
    vmin = np.amin(covid_active)
else:
    vmin = nor_min

if nor_max > np.amax(covid_active):
    vmax = np.amax(covid_active)
else:
    vmax = nor_max


vmin: 0.0
vmax: 1169803.0


In [141]:
model_coords = []
model_colors = []

# covid data
model_coords.extend([ coords_conversion(lon, lat, 0) for lon, lat, num in covid_coords ])
model_colors.extend([ point_color(num, vmin, vmax) for lon, lat, num in covid_coords ])

model('model.ply')

**3.3 display 3D model**

In [145]:
# read and display model
PC = o3d.io.read_point_cloud('model.ply')
o3d.visualization.draw_geometries([PC])